# Muti-Turn

In [ ]:
!pip install bitsandbytes==0.43.1
!pip install accelerate==0.30.1
!pip install transformers==4.39.3
!pip install gradio==4.29.0

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_id)

config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type='nf4',
  bnb_4bit_double_quant=True,
  bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    quantization_config=config
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
messages = [
    {"role": "system", "content": "You are a nice chatbot that helps users. You always have to respond briefly, within three sentences."},
    {"role": "user", "content": "What is the capital of the United States?"},
    {"role": "assistant", "content": "The capital of the United States is Washington D.C."},
    {"role": "user", "content": "Then, what about Korean?"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors='pt'
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
response = outputs[0][input_ids.shape[-1]:]

print("response: ", tokenizer.decode(response, skip_special_tokens=True))

response:  The capital of South Korea is Seoul, and the capital of North Korea is Pyongyang.


# Few Shot Learning

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a Korea robot that summarizes documents. You MUST answer in Korea"
    },
    {
        "role": "user",
        "content": """
        ###document: 기후 변화는 수십 년에서 수백만 년에 걸친 기간 동안의 기상 패턴의 통계적 분포에서 장기적인 변화를 의미합니다.
        이는 평균 기상 조건의 변화, 또는 평균 조건 주변의 기상 분포의 변화를 의미할 수 있습니다.
        또한 이것은 기온, 강수량, 또는 바람 패턴의 변화를 포함할 수 있습니다.
        """
    }
]

for message in messages:
  message['content'] = message['content'].replace("\n","")

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors='pt'
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

response = outputs[0][input_ids.shape[-1]:]

print("response : ", tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


response :  기후 변화는 기상 패턴의 통계적 분포에서 장기적인 변화를 의미하는 것입니다.


In [ ]:
messages = [
    {"role": "system", "content": "You are a korea robot that summarizes documents. You MUST answer in Korea"},
    {"role": "user", "content": """
    ###document: 에펠탑은 프랑스 파리의 샹 드 마르스에 위치한 철제 격자 탑입니다.
    이 탑은 그것을 설계하고 건설한 회사의 엔지니어인 구스타브 에펠타의 이름을 딴 것입니다.
    1887년부터 1889년까지 1889년 세계 박람회의 입구로 건설되었으며,
    처음에는 그 디자인을 이유로 프랑스의 주요 예술가들과 지식인들로부터 비판을 받았습니다.
    하지만 이제는 프랑스의 전세계적인 문화 아이콘 그리고 세계에서 가장 알아보기 쉬운 구조물 중 하나가 되었습니다.
    """},
    {"role": "assistant", "content": """에펠탑은 파리의 철제 격자 구조물로, 1887년부터 1889년까지 건설되었으며,
    이후로는 프랑스의 전 세계적으로 인식된 상징이 되었습니다."""},

    {"role": "user", "content": """
    ###document: 애플은 컴퓨터 소프트웨어와 온라인 서비스를 설계, 개발, 판매하는 미국의 다국적 기술 회사로,
     캘리포니아 주 쿠퍼티노에 본사를 두고 있습니다.
     애플은 아마존, 구글, 마이크로소프트, 페이스북과 함께 빅 테크 기술 회사로 간주됩니다.
    """},
    {"role": "assistant", "content": """애플은 소비자 전자 제품과 소프트웨어를 설계하고 판매하는 주요 기술 회사로,
    아마존, 구글, 마이크로소프트, 페이스북과 함께 빅 테크로 간주됩니다."""},

    {"role": "user", "content": """
    ###document: 기후 변화는 수십 년에서 수백만 년에 걸친 기간 동안의 기상 패턴의 통계적 분포에서 장기적인 변화를 의미합니다.
    이는 평균 기상 조건의 변화, 또는 평균 조건 주변의 기상 분포의 변화를 의미할 수 있습니다.
    또한 이것은 기온, 강수량, 또는 바람 패턴의 변화를 포함할 수 있습니다.
    """}
]

for message in messages:
  message['content'] = message['content'].replace("\n", "")

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors='pt'
).to(model.device)

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
response = outputs[0][input_ids.shape[-1]:]

print('response :', tokenizer.decode(response, skip_special_tokens=True))

response : 기후 변화는 장기적인 기상 패턴의 통계적 분포에서 평균 기상 조건의 변화 또는 기상 분포의 변화를 의미합니다.


In [ ]:
%%writefile chatbot.py
import gradio as gr
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",
    trust_remote_code=True,
    quantization_config=config
)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

messages = [
    {"role": "system",
    "content": "You are a korea robot that summarizes documents. You MUST answer in Korea"},
    {"role": "user",
    "content": """
    ###document: 에펠탑은 프랑스 파리의 샹 드 마르스에 위치한 철제 격자 탑입니다.
    이 탑은 그것을 설계하고 건설한 회사의 엔지니어인 구스타브 에펠타의 이름을 딴 것입니다.
    1887년부터 1889년까지 1889년 세계 박람회의 입구로 건설되었으며,
    처음에는 그 디자인을 이유로 프랑스의 주요 예술가들과 지식인들로부터 비판을 받았습니다.
    하지만 이제는 프랑스의 전세계적인 문화 아이콘 그리고 세계에서 가장 알아보기 쉬운 구조물 중 하나가 되었습니다.
    """},
    {"role": "assistant", "content": """에펠탑은 파리의 철제 격자 구조물로, 1887년부터 1889년까지 건설되었으며,
    이후로는 프랑스의 전 세계적으로 인식된 상징이 되었습니다."""},

    {"role": "user", "content": """
    ###document: 애플은 컴퓨터 소프트웨어와 온라인 서비스를 설계, 개발, 판매하는 미국의 다국적 기술 회사로,
     캘리포니아 주 쿠퍼티노에 본사를 두고 있습니다.
     애플은 아마존, 구글, 마이크로소프트, 페이스북과 함께 빅 테크 기술 회사로 간주됩니다.
    """},
    {"role": "assistant", "content": """애플은 소비자 전자 제품과 소프트웨어를 설계하고 판매하는 주요 기술 회사로,
    아마존, 구글, 마이크로소프트, 페이스북과 함께 빅 테크로 간주됩니다."""}
]

def chatbot(message, history):

    message_dict = {
        "role": "user",
        "content" : message
    }
    messages.append(message_dict)

    for message in messages:
        message["content"] = message["content"].replace("\n", "")

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    response = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(response, skip_special_tokens=True)

    return response

demo = gr.ChatInterface(fn=chatbot,title="Echo Bot")
demo.launch(share=True)


Writing chatbot.py
